# Parameter optimization

In [2]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'use_vol': [-1], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.55],
                            'second_candle': [0.5],
                            'third_candle': [0.5]
                            },
                'LinearReg': {
                            'timeperiod': np.arange(2, 102, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 50


  0%|                                                                                                                                                                                                   | 0/50 [00:00<?, ?it/s]


Load the datasets...
Binance
ByBit
BinanceFutures
ByBitPerpetual


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:08:58<00:00, 82.77s/it]


# Check local statistics

In [3]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
10,Pattern_LinearReg,0,1,6,0.55,0.50,0.5,10,0,2854,1.744267,2.100833,2124.137067,5995.778333,1.1888,1.1835,1.2361,1.3006,1.3980,1.4909,1.5247,1.5702,1.6131,1.6293,1.7794,1.8145,1.8543,1.8865,1.9188,1.9488,1.9830,2.0251,2.0333,2.0136,2.0315,2.1364,2.1446,2.1574,1.38,1.33,1.37,1.47,1.58,1.67,1.75,1.82,1.89,1.97,2.08,2.16,2.20,2.26,2.32,2.36,2.43,2.48,2.53,2.54,2.61,2.69,2.73,2.80
0,Pattern_LinearReg,0,1,6,0.50,0.50,0.5,10,0,3000,1.734404,2.097917,2203.212500,6293.750000,1.1897,1.1757,1.2224,1.2689,1.3820,1.4699,1.5109,1.5489,1.5970,1.6147,1.7593,1.7960,1.8374,1.8760,1.9202,1.9445,1.9828,2.0259,2.0311,2.0165,2.0319,2.1287,2.1397,2.1556,1.42,1.33,1.37,1.46,1.56,1.65,1.73,1.80,1.88,1.96,2.07,2.16,2.20,2.26,2.32,2.35,2.43,2.49,2.53,2.55,2.62,2.69,2.73,2.79
7,Pattern_LinearReg,0,1,6,0.50,0.85,0.5,10,0,1401,1.773362,2.097083,1083.480862,2938.013750,1.2370,1.1661,1.1921,1.2366,1.3395,1.4574,1.5074,1.5778,1.6114,1.6419,1.9093,1.9168,1.9701,1.9976,2.0374,2.0339,2.0477,2.0736,2.0823,2.0767,2.0951,2.1109,2.1211,2.1210,1.49,1.39,1.45,1.46,1.55,1.67,1.72,1.78,1.84,1.94,2.03,2.11,2.17,2.23,2.29,2.32,2.37,2.45,2.48,2.55,2.66,2.73,2.79,2.86
1,Pattern_LinearReg,0,1,6,0.50,0.55,0.5,10,0,2852,1.747250,2.097083,2131.157000,5980.881667,1.1801,1.1793,1.2227,1.2763,1.3867,1.4869,1.5320,1.5672,1.6093,1.6254,1.7781,1.8108,1.8571,1.8963,1.9406,1.9654,1.9994,2.0436,2.0447,2.0286,2.0407,2.1436,2.1535,2.1657,1.41,1.34,1.37,1.45,1.56,1.65,1.72,1.79,1.87,1.96,2.06,2.16,2.20,2.27,2.33,2.36,2.43,2.49,2.54,2.55,2.62,2.68,2.73,2.79
2,Pattern_LinearReg,0,1,6,0.50,0.60,0.5,10,0,2694,1.716479,2.091250,1930.194875,5633.827500,1.1681,1.1425,1.1787,1.2308,1.3426,1.4448,1.4846,1.5280,1.5674,1.5841,1.7431,1.7777,1.8236,1.8681,1.9151,1.9385,1.9722,2.0188,2.0232,2.0067,2.0200,2.1284,2.1398,2.1487,1.43,1.35,1.39,1.45,1.56,1.65,1.73,1.80,1.87,1.96,2.06,2.16,2.20,2.26,2.31,2.34,2.41,2.47,2.51,2.53,2.60,2.67,2.71,2.77
3,Pattern_LinearReg,0,1,6,0.50,0.65,0.5,10,0,2529,1.708992,2.087083,1793.039925,5278.233750,1.1602,1.1500,1.1826,1.2373,1.3460,1.4412,1.4856,1.5330,1.5700,1.5840,1.7514,1.7805,1.8235,1.8662,1.9183,1.9383,1.9737,2.0189,2.0265,2.0165,2.0308,2.0470,2.0622,2.0721,1.41,1.34,1.36,1.43,1.54,1.64,1.72,1.79,1.86,1.95,2.06,2.15,2.19,2.25,2.30,2.34,2.41,2.48,2.51,2.55,2.61,2.68,2.73,2.79
17,Pattern_LinearReg,0,1,6,0.55,0.85,0.5,10,0,1294,1.810175,2.078750,1048.366450,2689.902500,1.2769,1.1868,1.2158,1.2668,1.3534,1.4737,1.5320,1.6011,1.6328,1.6622,1.9517,1.9611,2.0177,2.0442,2.0844,2.0858,2.0923,2.1186,2.1319,2.1220,2.1433,2.1607,2.1671,2.1619,1.45,1.39,1.47,1.47,1.55,1.67,1.72,1.78,1.83,1.92,2.01,2.07,2.13,2.19,2.25,2.27,2.33,2.41,2.45,2.52,2.63,2.72,2.78,2.88
13,Pattern_LinearReg,0,1,6,0.55,0.65,0.5,10,0,2351,1.727775,2.077917,1710.999025,4885.182083,1.1755,1.1655,1.2044,1.2799,1.3763,1.4756,1.5078,1.5621,1.5948,1.6068,1.7803,1.8097,1.8535,1.8874,1.9258,1.9500,1.9818,2.0266,2.0363,2.0209,2.0404,2.0582,2.0710,2.0760,1.37,1.34,1.37,1.44,1.55,1.65,1.73,1.79,1.86,1.94,2.04,2.13,2.17,2.23,2.28,2.33,2.40,2.45,2.49,2.53,2.59,2.67,2.72,2.80
20,Pattern_LinearReg,0,1,6,0.60,0.50,0.5,1

In [3]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
13,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,28,0,1707,2.022646,2.424167,1745.656437,4138.052500,1.4020,1.4110,1.4634,1.5375,1.6011,1.6794,1.7809,1.8337,1.8613,1.9181,1.9187,1.9451,2.1618,2.1930,2.2270,2.2610,2.3112,2.3620,2.3874,2.4195,2.4413,2.4635,2.4673,2.4963,1.48,1.42,1.60,1.71,1.85,1.94,2.04,2.08,2.18,2.24,2.28,2.33,2.43,2.51,2.62,2.73,2.81,2.91,3.01,3.07,3.13,3.21,3.26,3.34
18,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,38,0,1782,2.018304,2.423750,1814.618025,4319.122500,1.3825,1.4166,1.4696,1.5430,1.6011,1.6790,1.7763,1.8335,1.8639,1.9182,1.9224,1.9502,2.1589,2.1918,2.2195,2.2504,2.3031,2.3508,2.3753,2.4090,2.4312,2.4535,2.4558,2.4837,1.48,1.43,1.62,1.73,1.87,1.96,2.05,2.10,2.20,2.25,2.28,2.34,2.44,2.51,2.61,2.71,2.79,2.89,2.99,3.06,3.11,3.19,3.24,3.32
15,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,32,0,1740,2.017583,2.415000,1770.595000,4202.100000,1.3856,1.4092,1.4596,1.5336,1.5965,1.6754,1.7765,1.8283,1.8555,1.9119,1.9146,1.9439,2.1587,2.1896,2.2222,2.2564,2.3065,2.3575,2.3831,2.4168,2.4363,2.4580,2.4590,2.4873,1.48,1.43,1.61,1.72,1.86,1.94,2.04,2.08,2.18,2.23,2.26,2.32,2.42,2.50,2.60,2.71,2.79,2.89,2.99,3.05,3.11,3.19,3.24,3.32
17,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,36,0,1768,2.017342,2.413333,1798.660067,4266.773333,1.3729,1.4054,1.4562,1.5355,1.5948,1.6740,1.7695,1.8274,1.8578,1.9120,1.9162,1.9436,2.1564,2.1910,2.2213,2.2562,2.3098,2.3584,2.3829,2.4168,2.4391,2.4615,2.4647,2.4928,1.49,1.43,1.61,1.72,1.85,1.94,2.04,2.08,2.18,2.23,2.26,2.32,2.42,2.50,2.60,2.71,2.79,2.88,2.98,3.05,3.11,3.19,3.23,3.31
12,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,26,0,1699,2.015608,2.420000,1725.518558,4111.580000,1.3954,1.4057,1.4573,1.5339,1.5974,1.6756,1.7797,1.8331,1.8612,1.9166,1.9163,1.9404,2.1602,2.1896,2.2236,2.2515,2.3014,2.3489,2.3732,2.4050,2.4262,2.4485,2.4524,2.4815,1.49,1.42,1.61,1.72,1.85,1.94,2.04,2.08,2.18,2.24,2.28,2.33,2.43,2.51,2.62,2.73,2.81,2.90,3.00,3.06,3.11,3.19,3.23,3.31
16,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,34,0,1753,2.014971,2.410000,1779.243871,4224.730000,1.3804,1.4093,1.4599,1.5337,1.5955,1.6737,1.7710,1.8259,1.8534,1.9082,1.9117,1.9397,2.1549,2.1893,2.2190,2.2531,2.3027,2.3540,2.3796,2.4132,2.4331,2.4551,2.4565,2.4864,1.49,1.44,1.62,1.72,1.85,1.94,2.04,2.08,2.17,2.22,2.25,2.31,2.41,2.49,2.60,2.70,2.78,2.88,2.98,3.05,3.10,3.18,3.23,3.31
14,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,30,0,1722,2.009771,2.413333,1738.825375,4155.760000,1.3851,1.4030,1.4561,1.5280,1.5912,1.6708,1.7695,1.8195,1.8480,1.9074,1.9075,1.9349,2.1504,2.1811,2.2137,2.2473,2.2966,2.3479,2.3731,2.4044,2.4259,2.4478,2.4483,2.4770,1.47,1.41,1.60,1.71,1.85,1.94,2.04,2.08,2.18,2.24,2.27,2.32,2.42,2.50,2.60,2.71,2.79,2.89,2.99,3.05,3.11,3.19,3.24,3.32
20,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,42,0,1820,2.005558,2.408333,1830.116167,4383.166667,1.3791,1.4106,1.4652,1.5414,1.5932,1.6708,1.7693,1.8265,1.8536,1.9071,1.9112,1.9413,2.1432,2.1750,2.2017,2.2360,2.2898,2.3332,2.3591,2.3901,2.4121,2.4319,2.4324,2.4596,1.48,1.42,1.61,1.72,1.86,1.94,2.04,2.08,2.18,2.23,2.26,2.32,2.43,2.50,2.60,2.70,2.78,2.88,2.98,3.04,3.09,3.16,3.21,3.29
19,Pattern_LinearReg,-1,1,5,0.55,0.5,

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_LinearReg,1,1,6,0.55,0.5,0.5,60,0,551,1.818942,1.995417,451.236858,1099.474583,1.1888,1.2354,1.3027,1.3673,1.5814,1.6774,1.6764,1.7264,1.7592,1.8528,1.9154,1.9542,1.9666,1.9982,2.0992,2.1174,2.1445,2.1514,2.0885,1.9851,1.9856,1.9784,1.9693,1.9330,1.21,1.25,1.36,1.43,1.58,1.66,1.62,1.63,1.69,1.81,1.91,1.96,1.99,2.07,2.15,2.23,2.33,2.37,2.45,2.48,2.59,2.65,2.72,2.75
0,Pattern_LinearReg,-1,1,6,0.55,0.5,0.5,60,0,2444,1.781908,2.169583,1910.983967,5302.461667,1.2078,1.2047,1.2522,1.3232,1.3863,1.4637,1.5139,1.5612,1.5867,1.6005,1.7719,1.8106,1.8541,1.9136,1.9552,1.9920,2.0260,2.0727,2.1017,2.1125,2.1431,2.2753,2.3046,2.3323,1.39,1.32,1.37,1.47,1.60,1.72,1.81,1.85,1.91,1.99,2.08,2.17,2.24,2.34,2.40,2.45,2.52,2.58,2.64,2.69,2.77,2.86,2.91,2.99
1,Pattern_LinearReg,0,1,6,0.55,0.5,0.5,60,0,3512,1.756362,2.121250,2656.345100,7449.830000,1.2116,1.1889,1.2439,1.3061,1.3923,1.4756,1.5190,1.5554,1.5894,1.6222,1.7630,1.8019,1.8397,1.8845,1.9350,1.9703,2.0061,2.0472,2.0658,2.0591,2.0845,2.1775,2.1980,2.2157,1.35,1.29,1.34,1.44,1.56,1.67,1.75,1.79,1.85,1.94,2.04,2.13,2.19,2.27,2.34,2.39,2.47,2.54,2.60,2.64,2.73,2.80,2.86,2.93


In [3]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
8,Pattern_LinearReg,-1,3,4,0.55,0.5,0.5,60,0,1167,1.868262,2.294167,1013.262337,2677.292500,1.1661,1.1483,1.1893,1.2642,1.3459,1.4771,1.5089,1.5425,1.5670,1.5901,1.8975,1.9649,2.0050,2.0656,2.1023,2.1328,2.1739,2.2186,2.2522,2.2711,2.2864,2.5273,2.5537,2.5876,1.43,1.35,1.49,1.56,1.65,1.78,1.91,1.94,1.99,2.08,2.22,2.32,2.41,2.50,2.52,2.58,2.65,2.72,2.79,2.86,2.98,3.04,3.11,3.18
9,Pattern_LinearReg,-1,3,5,0.55,0.5,0.5,60,0,1524,1.827888,2.288750,1261.700550,3488.055000,1.1306,1.1291,1.2017,1.2914,1.3745,1.4892,1.5268,1.5683,1.5876,1.6075,1.8540,1.9190,1.9432,1.9959,2.0399,2.0703,2.1137,2.1585,2.1721,2.1917,2.2088,2.4014,2.4320,2.4621,1.41,1.37,1.48,1.58,1.70,1.83,1.93,1.96,2.02,2.10,2.23,2.33,2.40,2.50,2.53,2.60,2.66,2.72,2.77,2.83,2.91,2.97,3.02,3.08
0,Pattern_LinearReg,-1,1,4,0.55,0.5,0.5,60,0,1795,1.797171,2.216250,1430.921646,3978.168750,1.2195,1.2132,1.2354,1.2928,1.3677,1.4528,1.5124,1.5440,1.5804,1.5924,1.8010,1.8289,1.8841,1.9439,1.9783,2.0098,2.0467,2.0945,2.1368,2.1444,2.1704,2.3323,2.3612,2.3892,1.43,1.33,1.40,1.49,1.61,1.71,1.84,1.87,1.93,2.01,2.12,2.21,2.29,2.40,2.45,2.49,2.56,2.64,2.71,2.77,2.86,2.95,3.02,3.10
1,Pattern_LinearReg,-1,1,5,0.55,0.5,0.5,60,0,2154,1.782758,2.215000,1686.061450,4771.110000,1.1849,1.1913,1.2422,1.3088,1.3838,1.4647,1.5239,1.5624,1.5952,1.6073,1.7889,1.8217,1.8636,1.9176,1.9572,1.9903,2.0277,2.0751,2.1025,2.1128,2.1380,2.2792,2.3106,2.3365,1.41,1.34,1.40,1.51,1.64,1.75,1.85,1.88,1.95,2.03,2.13,2.22,2.29,2.40,2.46,2.51,2.57,2.64,2.70,2.75,2.83,2.91,2.96,3.03
10,Pattern_LinearReg,-1,3,6,0.55,0.5,0.5,60,0,1805,1.817163,2.214167,1474.978313,3996.570833,1.1694,1.1587,1.2278,1.3185,1.3857,1.4899,1.5168,1.5694,1.5772,1.5969,1.8183,1.8872,1.9145,1.9736,2.0180,2.0557,2.0906,2.1328,2.1503,2.1690,2.1946,2.3688,2.3978,2.4304,1.38,1.33,1.42,1.51,1.63,1.77,1.86,1.90,1.96,2.03,2.14,2.24,2.31,2.39,2.43,2.50,2.57,2.64,2.68,2.74,2.83,2.90,2.96,3.02
5,Pattern_LinearReg,-1,2,5,0.55,0.5,0.5,60,0,2098,1.785025,2.193750,1646.982450,4602.487500,1.1878,1.1830,1.2233,1.2999,1.3791,1.4591,1.5225,1.5638,1.5944,1.6020,1.7924,1.8240,1.8683,1.9252,1.9604,1.9935,2.0314,2.0845,2.1077,2.1223,2.1480,2.2944,2.3237,2.3499,1.38,1.30,1.38,1.48,1.60,1.72,1.82,1.86,1.93,2.01,2.12,2.21,2.28,2.39,2.43,2.49,2.55,2.62,2.68,2.73,2.81,2.89,2.95,3.02
11,Pattern_LinearReg,-1,3,7,0.55,0.5,0.5,60,0,2118,1.854754,2.190833,1810.369325,4640.185000,1.2070,1.1689,1.2399,1.3200,1.3920,1.4863,1.5212,1.5720,1.5917,1.6260,1.8200,1.9009,1.9388,1.9932,2.0365,2.0763,2.1086,2.1468,2.2729,2.2898,2.3198,2.4665,2.4923,2.5267,1.34,1.28,1.37,1.46,1.58,1.72,1.82,1.86,1.93,2.01,2.11,2.21,2.28,2.36,2.40,2.48,2.56,2.63,2.68,2.74,2.84,2.91,2.97,3.04
4,Pattern_LinearReg,-1,2,4,0.55,0.5,0.5,60,0,1745,1.802171,2.187083,1399.788104,3816.460417,1.2239,1.2052,1.2174,1.2825,1.3606,1.4452,1.5098,1.5459,1.5820,1.5884,1.8075,1.8337,1.8921,1.9550,1.9836,2.0173,2.0544,2.1088,2.1470,2.1601,2.1864,2.3548,2.3810,2.4095,1.40,1.29,1.37,1.47,1.56,1.68,1.80,1.84,1.90,1.98,2.09,2.18,2.27,2.38,2.41,2.46,2.53,2.61,2.68,2.74,2.84,2.93,3.00,3.08
2,Pattern_LinearReg,-1,1,6,0.55,0.5,0.5,60,

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'use_vol': [-1], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.55],
                            'second_candle': [0.5],
                            'third_candle': [0.5]
                            },
                'LinearReg': {
                            'timeperiod': [60], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)